In [1]:
import openreview
import csv
import sys

client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net'
)
venue_id = 'ICLR.cc/2024/Conference'
venue_group = client.get_group(venue_id)
submission_name = venue_group.content['submission_name']['value']
submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}', details='replies')

Getting V2 Notes: 100%|█████████▉| 7396/7404 [00:32<00:00, 224.40it/s]


In [2]:
venue_group.content['review_name']['value']

'Official_Review'

In [27]:
len(submissions)

7404

In [51]:
review_name = venue_group.content['review_name']['value']

reviews = [
    {
        'submission_id': s.number,
        'summary': note.content['summary']['value'],
        'soundness': note.content['soundness']['value'],
        'presentation': note.content['presentation']['value'],
        'contribution': note.content['contribution']['value'],
        'strengths': note.content['strengths']['value'],
        'weaknesses': note.content['weaknesses']['value'],
        'questions': note.content['questions']['value'],
        'flag_for_ethics_review': note.content['flag_for_ethics_review']['value'],
        'rating': note.content['rating']['value'],
        'confidence': note.content['confidence']['value'],
        'code_of_conduct': note.content['code_of_conduct']['value']
    }
    for s in submissions
    for reply in s.details['replies']
    if f'{venue_id}/{submission_name}{s.number}/-/{review_name}' in reply['invitations']
    for note in [openreview.api.Note.from_json(reply)]
]

In [47]:
len(reviews)

28028

In [48]:
reviews[0]

{'submission_id': 1647,
 'review': {'summary': 'This work proposes LSTNet, a self-supervised method to establish reliable 3D dense correspondences irrespective of the input point clouds’ rotational orientation.\n\nSpecifically, LSTNet learns to formulate SO(3)-invariant local shape transform for each point in a dynamic, input-dependent manner. Each point-wise local shape transform can map the SO(3)-equivariant global shape descriptor of the input shape to a local shape descriptor, which is passed to the decoder to reconstruct the shape and pose of the input point cloud. \n\nThe proposed self-supervised training pipeline encourages semantically corresponding points from different shape instances to be mapped to similar local shape descriptors, enabling LSTNet to establish dense point-wise correspondences via nearest point pairs between cross-reconstructed point clouds.',
  'soundness': '3 good',
  'presentation': '2 fair',
  'contribution': '3 good',
  'strengths': 'The self- and cross-

In [52]:
# CSVファイルに保存する部分
csv_filename = 'iclr2024_reviews.csv'

with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = reviews[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for review in reviews:
        writer.writerow(review)

print(f"Reviews have been saved to {csv_filename}")

Reviews have been saved to iclr2024_reviews.csv


In [53]:
# pandas で読み込む
import pandas as pd
df = pd.read_csv(csv_filename)
df.head()

,submission_id,summary,soundness,presentation,contribution,strengths,weaknesses,questions,flag_for_ethics_review,rating,confidence,code_of_conduct
0,1647,"This work proposes LSTNet, a self-supervised m...",3 good,2 fair,3 good,The self- and cross-reconstruction training st...,The performance of aligned shape pairs under t...,The reason why other methods are much better t...,['No ethics review needed.'],6: marginally above the acceptance threshold,"2: You are willing to defend your assessment, ...",Yes
1,1647,1) This paper proposes a self-supervised metho...,3 good,3 good,2 fair,1) This paper is generally well-written;\n\n2)...,1) The main weakness of this paper could be al...,Please refer to the weaknesses.,['No ethics review needed.'],5: marginally below the acceptance threshold,"4: You are confident in your assessment, but n...",Yes
2,1647,"This paper introduces LSTNet, which leverages ...",2 fair,2 fair,2 fair,1. The idea of cross-reconstruction for genera...,1. The novelty of this work seems insufficient...,See weaknesses.,['No ethics review needed.'],"3: reject, not good enough","4: You are confident in your assessment, but n...",Yes
3,1647,This paper attempts to register point cloud pr...,3 good,3 good,3 good,"- Valid motivation. Unlike the abused topic, v...",My major concern is with the experimental setu...,"Following my points in the ""weaknesses"" sectio...",['No ethics review needed.'],5: marginally below the acceptance threshold,"4: You are confident in your assessment, but n...",Yes
4,1647,This paper presents a method of learning dense...,3 good,3 good,2 fair,1. The paper is in general well organized and ...,1. The main issue of the proposed method lies ...,Please refer to the Weaknees part.,['No ethics review needed.'],5: marginally below the acceptance threshold,"4: You are confident in your assessment, but n...",Yes
